# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [21]:
def square_root_generator(limit):
    n = 1
    sum_output = 0
    while n <= limit:
      sum_output = sum_output + n ** 0.5
      yield n ** 0.5, sum_output
      n += 1



# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)



(1.0, 1.0)
(1.4142135623730951, 2.414213562373095)
(1.7320508075688772, 4.146264369941973)
(2.0, 6.146264369941973)
(2.23606797749979, 8.382332347441762)


In [22]:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

(1.0, 1.0)
(1.4142135623730951, 2.414213562373095)
(1.7320508075688772, 4.146264369941973)
(2.0, 6.146264369941973)
(2.23606797749979, 8.382332347441762)
(2.449489742783178, 10.83182209022494)
(2.6457513110645907, 13.47757340128953)
(2.8284271247461903, 16.30600052603572)
(3.0, 19.30600052603572)
(3.1622776601683795, 22.4682781862041)
(3.3166247903554, 25.7849029765595)
(3.4641016151377544, 29.249004591697254)
(3.605551275463989, 32.854555867161245)


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [7]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [4]:
%%capture
!pip install "dlt[duckdb]" # Install dlt with all the necessary DuckDB dependencies

In [5]:
import dlt

# define the connection to load to.
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

In [10]:
#load people_1 generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707824544.986281 is LOADED and contains no failed jobs


In [12]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)

#print("\n\n\n people table below:")

#passengers = conn.sql("SELECT * FROM stream_download").df()
#display(passengers)

# As you can see, the same data was loaded in both cases.

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707824544.986281,WvwspvS3mKNpwA
1,2,Person_2,27,City_A,1707824544.986281,Lm9+3Q489wm0vQ
2,3,Person_3,28,City_A,1707824544.986281,8dfPeki5v/21uA
3,4,Person_4,29,City_A,1707824544.986281,PIQax7cjfTVsTg
4,5,Person_5,30,City_A,1707824544.986281,Mlgiw1o/83436A





 people table below:


In [13]:
#Append people_2 generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707824960.2967565 is LOADED and contains no failed jobs


In [14]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707824544.986281,WvwspvS3mKNpwA,None
1,2,Person_2,27,City_A,1707824544.986281,Lm9+3Q489wm0vQ,None
2,3,Person_3,28,City_A,1707824544.986281,8dfPeki5v/21uA,None
3,4,Person_4,29,City_A,1707824544.986281,PIQax7cjfTVsTg,None
4,5,Person_5,30,City_A,1707824544.986281,Mlgiw1o/83436A,None
5,3,Person_3,33,City_B,1707824960.2967565,t6hTHIpaEKrLug,Job_3
6,4,Person_4,34,City_B,1707824960.2967565,XAsgjA1t0RcibA,Job_4
7,5,Person_5,35,City_B,1707824960.2967565,hvl3YYKxq+u2xw,Job_5
8,6,Person_6,36,City_B,1707824960.2967565,wYfjbPjEpKc54Q,Job_6
9,7,Person_7,37,City_B,1707824960.2967565,D5X8CpXUsYHB1g,Job_7


from matplotlib import pyplot as plt
people['id'].plot(kind='hist', bins=20, title='id')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(people['city'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(people, x='id', y='city', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(people['_dlt_load_id'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(people, x='id', y='_dlt_load_id', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(people['occupation'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(people, x='id', y='occupation', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [16]:
people['age'].sum()

353

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [8]:
info = generators_pipeline.run(people_1(),
										table_name="people_new",
										write_disposition="replace",
                    primary_key="id")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707827873.8267784 is LOADED and contains no failed jobs


In [9]:
info = generators_pipeline.run(people_2(),
										table_name="people_new",
										write_disposition="merge",
                    primary_key="id")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.65 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707827913.6510453 is LOADED and contains no failed jobs


In [10]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n  people new table below:")



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_new          │
└─────────────────────┘




  people new table below:


NameError: name 'people_new' is not defined

In [11]:
people_new = conn.sql("SELECT * FROM people_new").df()
display(people_new)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707827873.8267784,PceHz7wLtdYdfw,None
1,2,Person_2,27,City_A,1707827873.8267784,ziNfmrGuLMY0aw,None
2,8,Person_8,38,City_B,1707827913.6510453,JlvPS8QxFaC0KQ,Job_8
3,4,Person_4,34,City_B,1707827913.6510453,ugPbxhXsdk5BFw,Job_4
4,5,Person_5,35,City_B,1707827913.6510453,t4iEu8tzB52fQg,Job_5
5,7,Person_7,37,City_B,1707827913.6510453,YrMtGRWSSfTxPA,Job_7
6,3,Person_3,33,City_B,1707827913.6510453,zA3NIzgKFJG3eA,Job_3
7,6,Person_6,36,City_B,1707827913.6510453,WsT2ScK98+O3rw,Job_6


In [12]:
people_new['age'].sum()

266

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX